## 07. 커스텀 데이터셋(Custom Dataset)

앞 내용을 잠깐 복습해보자. 파이토치에서는 데이터셋을 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 torch.utils.data.Dataset과 torch.utils.data.DataLoader를 제공한다. 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있다. 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것이다.

### 1. 커스텀 데이터셋(Custom Dataset)

그런데 torch.utils.data.Dataset을 상속받아 직접 커스텀 데이터셋(Custom Dataset)을 만드는 경우도 있다. torch.utils.data.Dataset은 파이토치에서 데이터셋을 제공하는 추상 클래스이다. Dataset을 상속받아 다음 메소드들을 오버라이드 하여 커스텀 데이터셋을 만들어보겠다.

커스텀 데이터셋을 만들 때, 일단 가장 기본적인 뼈대는 아래와 같다. 여기서 필요한 기본적인 define은 3개이다.

``` python
class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):

  def __len__(self):

  def __getitem__(self, idx): 
```

이를 좀 더 자세히 보자.

``` python
class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):
  데이터셋의 전처리를 해주는 부분

  def __len__(self):
  데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

  def __getitem__(self, idx): 
  데이터셋에서 특정 1개의 샘플을 가져오는 함수
```

* len(dataset)을 했을 때 데이터셋의 크기를 리턴할 **len**

* dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 **get_item**

### 2. 커스텀 데이터셋(Custom Dataset)으로 선형 회귀 구현하기

In [1]:
import torch
import torch.nn.functional as F

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
# Dataset 상속
class CustomDataset(Dataset):

    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [98, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

    
    # 총 데이터의 개수를 리턴
    def __len__(self):
        return len(self.x_data)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [4]:
dataset = CustomDataset()
print(dataset)
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)
print(dataloader)

In [5]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
print(model)
print(optimizer)

Linear(in_features=3, out_features=1, bias=True)
SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-05
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [6]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 36023.503906
Epoch    0/20 Batch 2/3 Cost: 12063.558594
Epoch    0/20 Batch 3/3 Cost: 4208.456543
Epoch    1/20 Batch 1/3 Cost: 1197.227173
Epoch    1/20 Batch 2/3 Cost: 201.089355
Epoch    1/20 Batch 3/3 Cost: 50.726711
Epoch    2/20 Batch 1/3 Cost: 33.648499
Epoch    2/20 Batch 2/3 Cost: 11.300299
Epoch    2/20 Batch 3/3 Cost: 17.486502
Epoch    3/20 Batch 1/3 Cost: 4.691053
Epoch    3/20 Batch 2/3 Cost: 4.464823
Epoch    3/20 Batch 3/3 Cost: 3.395364
Epoch    4/20 Batch 1/3 Cost: 4.108072
Epoch    4/20 Batch 2/3 Cost: 5.489243
Epoch    4/20 Batch 3/3 Cost: 2.464540
Epoch    5/20 Batch 1/3 Cost: 2.711354
Epoch    5/20 Batch 2/3 Cost: 10.752176
Epoch    5/20 Batch 3/3 Cost: 3.500649
Epoch    6/20 Batch 1/3 Cost: 5.476945
Epoch    6/20 Batch 2/3 Cost: 4.446768
Epoch    6/20 Batch 3/3 Cost: 1.689691
Epoch    7/20 Batch 1/3 Cost: 2.171490
Epoch    7/20 Batch 2/3 Cost: 7.694203
Epoch    7/20 Batch 3/3 Cost: 1.457845
Epoch    8/20 Batch 1/3 Cost: 2.873114
Epoc

In [7]:
# 임의의 입력 [73, 80, 75]를 선언
new_var = torch.FloatTensor([[73, 80, 75]])

# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print('훈련 후 입력이 73, 80, 75일 때의 예측값 :', pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.2308]], grad_fn=<AddmmBackward>)
